In [72]:
# !pip install nltk
# !python -m nltk.downloader reuters


In [73]:
import nltk
from nltk.util import everygrams
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import reuters
import re
import contractions

In [74]:
SENTENCE = "today the company"

In [75]:
def expand_contractions(text):
    return contractions.fix(text)

def preprocess(text):
    text = expand_contractions(text)
    text = text.lower()
    return text

def generate_sentence(input_sentence, model):
    for _ in range(1):  # generate one word
        next_word_candidates = model[input_sentence]
        while not next_word_candidates and input_sentence:
            input_sentence = ' '.join(input_sentence.split(' ')[1:])
            next_word_candidates = model[input_sentence]
        if not next_word_candidates:
            break
        input_sentence = max(next_word_candidates, key=next_word_candidates.get)
    return input_sentence

In [76]:
# Tokenize the reuters corpus
tokens = word_tokenize(preprocess(reuters.raw()))
#print(reuters.raw())

In [77]:
# Create n-grams where n is the number of words in the input sentence
n = len(SENTENCE.split())
ngrams = list(everygrams(tokens, max_len=n+1))

# Initialize the model
model = defaultdict(Counter)

In [78]:
# Count frequency of co-occurence
for ngram in ngrams:
    if len(ngram) == n+1:
        *init_words, next_word = ngram
        init_words = ' '.join(init_words)
        model[init_words][next_word] += 1

# Let's transform the counts to probabilities
for w1 in model:
    total_count = float(sum(model[w1].values()))
    for w2 in model[w1]:
        model[w1][w2] /= total_count

In [79]:
# Generate a sentence
ans = generate_sentence(SENTENCE, model)
print(f"{SENTENCE} <{ans}>")


today the company <reported>
